### 2D Platformer<br>

***

<br>

    ServerStorage / Workspace (local)
        scripts - Folder
            BombScript - Script
                Sonic Explosion Sound - Sound
            CoinScript - Script
                Coin - Sound
            DamageScript - Script
            MovementScript - Script
                WalkAnim - Animation
            MysteryScript - Script
                Mystery - Sound
            SpinBoxScript - Script
                Mystery - Sound
            SpinCoinScript - Script
                Coin - Sound
            TrapScript - Script
            Wizard Script - Script
                Animation - Animation

Animation ID: rbxassetid://7214822129

In [ ]:
# Wizard Script
local wizard = script.Parent:WaitForChild("Humanoid")
local Debris = game:GetService("Debris")
local spellAnimTrack = wizard:LoadAnimation(script.Animation)
spellAnimTrack.Looped = false

local function onDeath()
	local bridge = script.Parent.Parent.Victory
	bridge.Transparency = 0
	bridge.CanCollide = true
	
	local block = script.Parent.Parent.BlockPath
	block.Transparency = 1
	block.CanCollide = false
	
	local GbLb = game.Workspace.GlobalLeaderboard
	GbLb.Transparency = 0
	local GbChild = GbLb.LeaderboardGUI
	GbChild.Holder.Visible = true
	GbChild.TopBar.Visible = true
	
	local HiLb = game.Workspace.HiScoreLeaderboard
	HiLb.Transparency = 0
	local HiChild = HiLb.LeaderboardGUI
	HiChild.Holder.Visible = true
	HiChild.TopBar.Visible = true

end
wizard.Died:Connect(onDeath)

local function spellLanded(hit, spell)
	if hit.Parent:FindFirstChild("Humanoid") then
		spell:Destroy()
		local character = hit.Parent
		if character:FindFirstChild("enchanted") and character.enchanted.Value == true then
			wizard.Health = wizard.Health - (wizard.MaxHealth / 3)
			character.enchanted.Value = false
		else
			character.Humanoid.Health = character.Humanoid.Health - (character.Humanoid.MaxHealth / 3)
			
			local force = Instance.new("BodyForce")
			force.Force = Vector3.new(-500, 500, 0)
			force.Parent = character.Humanoid.RootPart
			character.Humanoid.Sit = true
			wait(2)
			force:Destroy()
		end
	end
end

local function CreateFirebolt()
	local firebolt = Instance.new("Part")
	firebolt.Position = wizard.Parent.HumanoidRootPart.Position + Vector3.new(-2, 0, 0)
	firebolt.Shape = "Ball"
	firebolt.Size = Vector3.new(1,1,1)
	firebolt.BrickColor = BrickColor.Red()
	firebolt.CanCollide = false
	firebolt.Parent = workspace
	
	firebolt.Touched:Connect(function(hit)
		spellLanded(hit, firebolt)
	end)
	
	local fire = Instance.new("Fire", firebolt)
	
	local force = Instance.new("BodyForce")
	force.Force = Vector3.new(-10, 72, 0)
	force.Parent = firebolt
	
	Debris:AddItem(firebolt, 3)
end

wait(2)
while wizard.Health > 0 do
	spellAnimTrack:Play()
	CreateFirebolt()
	wait(2)
end

    ServerScriptService - ServerScriptService
        PlayerSetup - Script
        TimeGain - Script
        TrailScript - Script
        LeaderboardHandler - Script
        HiScoreLbHandler - Script
        DeathCounter - Script

This helps with initial player setup, when player loads into the game<br>
important part is that ServerStorage maps and scripts are loaded to workspace<br>
rather than workspace containing any of actual files<br>

In [ ]:
# PlayerSetup
local DataStoreService = game:GetService("DataStoreService")
local Hiscore = DataStoreService:GetDataStore("HiScore")
local Totaltime = DataStoreService:GetDataStore("TotalTime")
local Totalpoints = DataStoreService:GetDataStore("TotalPoints")
local Totalwins = DataStoreService:GetDataStore("TotalWins")
local Totaldeath = DataStoreService:GetDataStore("TotalDeath")

local function CleanUp()
	for key, object in pairs(workspace:GetChildren()) do
		if object:IsA("Part") or object:IsA("Model") then
			object:Destroy()
		end
	end
end

local function GetContent()
	CleanUp()
	local ServerStorage = game:GetService("ServerStorage")
	for key, object in pairs(ServerStorage.map:GetChildren()) do
		local clone = object:Clone()
		clone.Parent = workspace
	end
end

game.Players.PlayerAdded:connect(function(player)
	GetContent()
	player.RespawnLocation = workspace.level1.level1Spawn

	local key = "user_"..player.userId
	local leaderstats = Instance.new("Folder")
	leaderstats.Name = "leaderstats"
	leaderstats.Parent = player

	local level = Instance.new("IntValue")
	level.Name = "Level"
	level.Parent = leaderstats
	level.Value = 1	

	local points = Instance.new("IntValue")
	points.Name = "Points"
	points.Parent = leaderstats
	points.Value = 0

	local hiscore = Instance.new("IntValue")
	hiscore.Name = "HiScore"
	hiscore.Parent = leaderstats
	hiscore.Value = Hiscore:GetAsync(key) or 0
	
	-- hidden stats
	local storage = Instance.new("Folder")
	storage.Name = "Storage"
	storage.Parent = player
	
	local totaltime = Instance.new("IntValue")
	totaltime.Name = "TotalTime"
	totaltime.Parent = storage
	totaltime.Value = Totaltime:GetAsync(key) or 0

	local totalpoints = Instance.new("IntValue")
	totalpoints.Name = "TotalPoints"
	totalpoints.Parent = storage
	totalpoints.Value = Totalpoints:GetAsync(key) or 0

	local totalwins = Instance.new("IntValue")
	totalwins.Name = "TotalWins"
	totalwins.Parent = storage
	totalwins.Value = Totalwins:GetAsync(key) or 0
	
	local totaldeath = Instance.new("IntValue")
	totaldeath.Name = "TotalDeath"
	totaldeath.Parent = storage
	totaldeath.Value = Totaldeath:GetAsync(key) or 0
	
	totaltime.Changed:connect(function()
		Totaltime:SetAsync(key, totaltime.Value)
		Totalpoints:SetAsync(key, totalpoints.Value)
		Totaldeath:SetAsync(key, totaldeath.Value)
		Totalwins:SetAsync(key, totalwins.Value)
		Hiscore:SetAsync(key, hiscore.Value)
	end)
end)

----
game.Players.PlayerRemoving:connect(function(player)
	local key = "user_"..player.userId
	local setSuccess, errorMessage = pcall(function()
		Totaltime:SetAsync(key, player.Storage.TotalTime.Value)
		Totalpoints:SetAsync(key, player.Storage.TotalPoints.Value)
		Totaldeath:SetAsync(key, player.Storage.TotalDeath.Value)
		Totalwins:SetAsync(key, player.Storage.TotalWins.Value)
		Hiscore:SetAsync(key, player.leaderstats.HiScore.Value)
	end)
	if not setSuccess then
		warn(errorMessage)
	end
	print("exit data was saved")
end)
----

local function RestartGame(player)
	GetContent()
	
	player.Storage.TotalWins.Value = player.Storage.TotalWins.Value + 1
	if player.leaderstats.Points.Value > player.leaderstats.HiScore.Value then
		player.leaderstats.HiScore.Value = player.leaderstats.Points.Value
	end
	
	local key = "user_"..player.userId
	local setSuccess, errorMessage = pcall(function()
		Totaltime:SetAsync(key, player.Storage.TotalTime.Value)
		Totalpoints:SetAsync(key, player.Storage.TotalPoints.Value)
		Totaldeath:SetAsync(key, player.Storage.TotalDeath.Value)
		Totalwins:SetAsync(key, player.Storage.TotalWins.Value)
		Hiscore:SetAsync(key, player.leaderstats.HiScore.Value)
	end)
	if not setSuccess then
		warn(errorMessage)
	end
	print("reset data was saved")	
	
	player.leaderstats.Points.Value = 0
	player.leaderstats.Level.Value = 1
	player.RespawnLocation = workspace.level1.level1Spawn
	player:LoadCharacter()
end

local RestartGameEvent = game:GetService("ReplicatedStorage"):WaitForChild("RestartGame")
RestartGameEvent.OnServerEvent:Connect(RestartGame)